<a href="https://colab.research.google.com/github/21002572uhi/21002572_DataAnalytics/blob/main/21002572_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Introduction

Previously, a task was undertaken to analyse the relationships between the weather and the number of traffic collisions in New York City.
From that analysis it was found that collision numbers tended to increase over the course of the weekdays, falling lower on Saturdays and were at their lowest on Sundays.

With regard to the weather factors, strong positive correlations between collisions and temperature were found, dewpoint was also positive and promising in some years. Correlations with other weather factors showed these to be less useful. There was also evidence of some seasonal patterns.
One day of the week data was reordered to create a more linear relationship with collision numbers, stronger correlations were produced, particularly once outliers were removed.

The knowledge developed via the previous task will now be used to support the development of linear regression and deep neural network (DNN) regression models, for predicting the number of collisions which would occur on a particular date.


# 2 Method

Firstly, the data is loaded, then cleansed in line with the findings in assignment 1, and then normalised. This is in preparation for training and testing first a linear regressor and then a deep neural network regressor. These steps are taken below.

### 2.1 Load data

In [430]:
# needed to create the data frame
import pandas as pd

# needed to help with speedy maths based calculations
import numpy as np

# needed for machine learning
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.15.0


In [431]:
# create data frame from csv file hosted on our github
df = pd.read_csv('https://raw.githubusercontent.com/21002572uhi/21002572_DataAnalytics/main/collated_collision_data_ordered.csv')

# set pandas display options to allow all columns to be shown on screen
pd.set_option('display.max_columns', None)

In [432]:
# check dataframe
df

,day,year,mo,da,collision_date,temp,dewp,slp,visib,wdsp,mxpsd,gust,max,min,prcp,sndp,fog,NUM_COLLISIONS
0,7,2012,7,1,2012-07-01,83.6,63.0,1008.9,9.7,4.1,9.9,18.1,93.0,66.0,0.00,999.9,0,538
1,1,2012,7,2,2012-07-02,80.3,54.1,1011.6,10.0,3.8,15.0,999.9,88.0,66.9,0.00,999.9,0,564
2,2,2012,7,3,2012-07-03,79.8,56.7,1012.8,10.0,2.9,12.0,999.9,88.0,63.0,0.00,999.9,0,664
3,3,2012,7,4,2012-07-04,81.8,65.6,1009.1,9.2,3.6,11.1,999.9,91.0,68.0,0.06,999.9,0,432
4,4,2012,7,5,2012-07-05,86.7,64.3,1007.4,9.4,3.8,15.0,999.9,93.9,70.0,99.99,999.9,0,591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3101,7,2020,12,27,2020-12-27,34.2,21.2,1021.2,10.0,7.7,15.0,22.0,44.1,28.9,0.00,999.9,0,184
3102,1,2020,12,28,2020-12-28,43.0,36.2,1024.8,10.0,9.5,14.0,999.9,52.0,26.1,0.00,999.9,0,217
3103,2,2020,12,29,2020-12-29,41.3,26.2,1022.0,9.7,13.0,19.0,28.0,52.0,32.0,0.00,999.9,0,244
3104,3,2020,12,30,2020-12-30,34.1,17.0,1029.9,10.0,11.0,19.0,24.1,48.0,25.0,0.00,999.9,0,238


### 2.2 Cleanse data

In the data analysis in assignment 1 it was realised that the collision numbers in 2020 were highly different to previous years, firstly plummeting in March 2020 when Covid lockdown occurred then recovering to lower numbers than in previous years, but with the previous near-reliable weekly and seasonal patterns now absent. 2020 is therefore to be removed before moving on to the regression steps.

In [433]:
# delete data from 2020 as COVID-19 pandemic has skewed results

prelen = len(df)
df_cleansed = df[df['year'] != 2020]
postlen = len(df_cleansed)
print("All " + str(prelen - postlen) + " rows deleted for 2020")

All 366 rows deleted for 2020


The next step is to remove the outliers that were identified in assignment 1. The outlier thresholds vary by year because there was a general upward trend year on year, from 2012 to 2018 and a fall in 2019 to below 2015 levels (though the general pattern still held in 2019).

In [434]:
# remove outliers as identified in assignment 1

print("Number of rows in dataframe before removing outliers = " + str(len(df_cleansed)))


# for each Year, NUM_ROWS is:
#
# 2012 >= 380
# 2012 <= 720
prelen = len(df_cleansed)
df_cleansed = df_cleansed.drop(df_cleansed[(df_cleansed.year == 2012) & (df_cleansed.NUM_COLLISIONS < 380)].index)
df_cleansed = df_cleansed.drop(df_cleansed[(df_cleansed.year == 2012) & (df_cleansed.NUM_COLLISIONS > 720)].index)
postlen = len(df_cleansed)
print(str(prelen - postlen) + " outliers deleted for 2012")
#
# 2013 >= 390
# 2013 <= 720
prelen = len(df_cleansed)
df_cleansed = df_cleansed.drop(df_cleansed[(df_cleansed.year == 2013) & (df_cleansed.NUM_COLLISIONS < 390)].index)
df_cleansed = df_cleansed.drop(df_cleansed[(df_cleansed.year == 2013) & (df_cleansed.NUM_COLLISIONS > 720)].index)
postlen = len(df_cleansed)
print(str(prelen - postlen) + " outliers deleted for 2013")
#
# 2014 >= 380
# 2014 <= 750
prelen = len(df_cleansed)
df_cleansed = df_cleansed.drop(df_cleansed[(df_cleansed.year == 2014) & (df_cleansed.NUM_COLLISIONS < 380)].index)
df_cleansed = df_cleansed.drop(df_cleansed[(df_cleansed.year == 2014) & (df_cleansed.NUM_COLLISIONS > 750)].index)
postlen = len(df_cleansed)
print(str(prelen - postlen) + " outliers deleted for 2014")
#
# 2015 >= 390
# 2015 <= 800
prelen = len(df_cleansed)
df_cleansed = df_cleansed.drop(df_cleansed[(df_cleansed.year == 2015) & (df_cleansed.NUM_COLLISIONS < 380)].index)
df_cleansed = df_cleansed.drop(df_cleansed[(df_cleansed.year == 2015) & (df_cleansed.NUM_COLLISIONS > 800)].index)
postlen = len(df_cleansed)
print(str(prelen - postlen) + " outliers deleted for 2015")
#
# 2016 >= 430
# 2016 <= 820
prelen = len(df_cleansed)
df_cleansed = df_cleansed.drop(df_cleansed[(df_cleansed.year == 2016) & (df_cleansed.NUM_COLLISIONS < 430)].index)
df_cleansed = df_cleansed.drop(df_cleansed[(df_cleansed.year == 2016) & (df_cleansed.NUM_COLLISIONS > 820)].index)
postlen = len(df_cleansed)
print(str(prelen - postlen) + " outliers deleted for 2016")
#
# 2017 >= 430
# 2017 <= 840
prelen = len(df_cleansed)
df_cleansed = df_cleansed.drop(df_cleansed[(df_cleansed.year == 2017) & (df_cleansed.NUM_COLLISIONS < 430)].index)
df_cleansed = df_cleansed.drop(df_cleansed[(df_cleansed.year == 2017) & (df_cleansed.NUM_COLLISIONS > 840)].index)
postlen = len(df_cleansed)
print(str(prelen - postlen) + " outliers deleted for 2017")
#
# 2018 >= 430
# 2018 <= 840
prelen = len(df_cleansed)
df_cleansed = df_cleansed.drop(df_cleansed[(df_cleansed.year == 2018) & (df_cleansed.NUM_COLLISIONS < 430)].index)
df_cleansed = df_cleansed.drop(df_cleansed[(df_cleansed.year == 2018) & (df_cleansed.NUM_COLLISIONS > 840)].index)
postlen = len(df_cleansed)
print(str(prelen - postlen) + " outliers deleted for 2018")
#
# 2019 >= 400
# 2019 <= 760
prelen = len(df_cleansed)
df_cleansed = df_cleansed.drop(df_cleansed[(df_cleansed.year == 2019) & (df_cleansed.NUM_COLLISIONS < 400)].index)
df_cleansed = df_cleansed.drop(df_cleansed[(df_cleansed.year == 2019) & (df_cleansed.NUM_COLLISIONS > 760)].index)
postlen = len(df_cleansed)
print(str(prelen - postlen) + " outliers deleted for 2019")
#
print("Number of rows in dataframe after removing outliers = " + str(len(df_cleansed)))



Number of rows in dataframe before removing outliers = 2740
11 outliers deleted for 2012
13 outliers deleted for 2013
12 outliers deleted for 2014
12 outliers deleted for 2015
14 outliers deleted for 2016
16 outliers deleted for 2017
13 outliers deleted for 2018
14 outliers deleted for 2019
Number of rows in dataframe after removing outliers = 2635


In [435]:
# create two new dataframes to hold error values and error counts

# copy df_cleansed to errs, removing unwanted columns
errs = df_cleansed.drop(['day','year','mo','da','collision_date','NUM_COLLISIONS'], axis=1)

# empty errs
errs = errs.iloc[0:0]

# copy errs to arrcount
errcount = errs
# errs <- subset(collcleanse, 1==2, select = -c(1,2,3,4,5,18))


The next step is to look at the missing values.

In [436]:
errs.at[1,'temp']=9999.9
errs.at[1,'dewp']=9999.9
errs.at[1,'slp']=9999.9
errs.at[1,'visib']=999.9
errs.at[1,'wdsp']=999.9
errs.at[1,'mxpsd']=999.9
errs.at[1,'gust']=9999.9
errs.at[1,'max']=9999.9
errs.at[1,'min']=9999.9
errs.at[1,'prcp']=99.9
errs.at[1,'sndp']=999.9
errs.at[1,'fog']=0

errs

,temp,dewp,slp,visib,wdsp,mxpsd,gust,max,min,prcp,sndp,fog
1,9999.9,9999.9,9999.9,999.9,999.9,999.9,9999.9,9999.9,9999.9,99.9,999.9,0.0


In [437]:


errcount.at[1,'temp'] = (df_cleansed['temp'] == errs['temp'].iloc[0]).sum()
errcount.at[1,'dewp'] = (df_cleansed['dewp'] == errs['dewp'].iloc[0]).sum()
errcount.at[1,'slp'] = (df_cleansed['slp'] == errs['slp'].iloc[0]).sum()
errcount.at[1,'visib'] = (df_cleansed['visib'] == errs['visib'].iloc[0]).sum()
errcount.at[1,'wdsp'] = (df_cleansed['wdsp'] == errs['wdsp'].iloc[0]).sum()
errcount.at[1,'mxpsd'] = (df_cleansed['mxpsd'] == errs['mxpsd'].iloc[0]).sum()
errcount.at[1,'gust'] = (df_cleansed['gust'] == errs['gust'].iloc[0]).sum()
errcount.at[1,'max'] = (df_cleansed['max'] == errs['max'].iloc[0]).sum()
errcount.at[1,'min'] = (df_cleansed['min'] == errs['min'].iloc[0]).sum()
errcount.at[1,'prcp'] = (df_cleansed['prcp'] == errs['prcp'].iloc[0]).sum()
errcount.at[1,'sndp'] = (df_cleansed['sndp'] == errs['sndp'].iloc[0]).sum()
errcount.at[1,'fog'] = (df_cleansed['fog'] == errs['fog'].iloc[0]).sum()

errcount

,temp,dewp,slp,visib,wdsp,mxpsd,gust,max,min,prcp,sndp,fog
1,0.0,1.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2634.0,1979.0


The number of missing values of dew point, sea level pressure and maximum sustained windspeed are low, so these can be removed so they don't cause issues.

In [438]:
# remove errors for DEWP
df_cleansed = df_cleansed[df_cleansed.dewp != 9999.9]
# remove errors for SLP
df_cleansed = df_cleansed[df_cleansed.slp != 9999.9]
# remove errors for MXPSD
df_cleansed = df_cleansed[df_cleansed.mxpsd != 999.9]

The next step is to reorder the days so that they are roughly aligned with the linear relationship observed in assignment 1, where the lowest number of collisions are on Sundays, then Saturdays, then rising through the weekdays to the highest number of collisions, on average, on Fridays.

In [439]:
# update values for day of the week such that:
# 1 = Sunday
# 2 = Saturday
# 3 = Monday
# 4 = Tuesday
# 5 = Wednesday
# 6 = Thursday
# 7 = Friday

df_cleansed['day'] = df_cleansed['day'] + 2

df_cleansed['day'] = df_cleansed['day'].replace(8,2)
df_cleansed['day'] = df_cleansed['day'].replace(9,1)


In [440]:
# replace df with cleansed data

df = df_cleansed

df

,day,year,mo,da,collision_date,temp,dewp,slp,visib,wdsp,mxpsd,gust,max,min,prcp,sndp,fog,NUM_COLLISIONS
0,1,2012,7,1,2012-07-01,83.6,63.0,1008.9,9.7,4.1,9.9,18.1,93.0,66.0,0.00,999.9,0,538
1,3,2012,7,2,2012-07-02,80.3,54.1,1011.6,10.0,3.8,15.0,999.9,88.0,66.9,0.00,999.9,0,564
2,4,2012,7,3,2012-07-03,79.8,56.7,1012.8,10.0,2.9,12.0,999.9,88.0,63.0,0.00,999.9,0,664
3,5,2012,7,4,2012-07-04,81.8,65.6,1009.1,9.2,3.6,11.1,999.9,91.0,68.0,0.06,999.9,0,432
4,6,2012,7,5,2012-07-05,86.7,64.3,1007.4,9.4,3.8,15.0,999.9,93.9,70.0,99.99,999.9,0,591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2732,4,2019,12,24,2019-12-24,42.8,33.7,1015.2,10.0,12.0,18.1,22.0,50.0,34.0,0.00,999.9,0,530
2734,6,2019,12,26,2019-12-26,35.9,30.3,1025.7,10.0,5.9,14.0,999.9,46.4,28.9,0.00,999.9,0,414
2735,7,2019,12,27,2019-12-27,40.1,35.0,1026.1,10.0,7.6,12.0,999.9,48.0,28.9,0.00,999.9,0,448
2738,3,2019,12,30,2019-12-30,39.6,36.3,1017.1,5.2,13.0,25.1,35.0,45.0,30.9,0.39,999.9,0,518


### 2.3 Normalise data

Next, the collision numbers are normalised and standardised for the whole dataset, and new columns are added with these figures.

In [441]:
# normalise the entire dataframe

# add new column normalising NUM_COLLISIONS against all data
df['norm_all'] = ((df['NUM_COLLISIONS'] - df['NUM_COLLISIONS'].min()) / (df['NUM_COLLISIONS'].max() - df['NUM_COLLISIONS'].min()))

# add new column standardising NUM_COLLISIONS against all data
df['stan_all'] = ((df['NUM_COLLISIONS'] - df['NUM_COLLISIONS'].mean()) / (df['NUM_COLLISIONS'].std()))

df[:5]

,day,year,mo,da,collision_date,temp,dewp,slp,visib,wdsp,mxpsd,gust,max,min,prcp,sndp,fog,NUM_COLLISIONS,norm_all,stan_all
0,1,2012,7,1,2012-07-01,83.6,63.0,1008.9,9.7,4.1,9.9,18.1,93.0,66.0,0.00,999.9,0,538,0.344227,-0.664699
1,3,2012,7,2,2012-07-02,80.3,54.1,1011.6,10.0,3.8,15.0,999.9,88.0,66.9,0.00,999.9,0,564,0.400871,-0.374090
2,4,2012,7,3,2012-07-03,79.8,56.7,1012.8,10.0,2.9,12.0,999.9,88.0,63.0,0.00,999.9,0,664,0.618736,0.743637
3,5,2012,7,4,2012-07-04,81.8,65.6,1009.1,9.2,3.6,11.1,999.9,91.0,68.0,0.06,999.9,0,432,0.113290,-1.849489
4,6,2012,7,5,2012-07-05,86.7,64.3,1007.4,9.4,3.8,15.0,999.9,93.9,70.0,99.99,999.9,0,591,0.459695,-0.072304


Next, normalised and standardised figures are calculated within each year and again added as two new columns.

In [442]:
# normalise NUM_COLLISIONS by year

# add new columns and set all values to zero
df['norm_yr'] = 0
df['stan_yr'] = 0

# create new list 'year_list' of all unique years from 'collcleanse' dataframe
year_list = df['year'].unique()

# loop through all years in list setting 'norm_yr' and 'stan_yr' values
for yr in year_list:
  setyear = yr
  maxcols = (df[df['year'] == setyear])['NUM_COLLISIONS'].max()
  mincols = (df[df['year'] == setyear])['NUM_COLLISIONS'].min()
  meancols = (df[df['year'] == setyear])['NUM_COLLISIONS'].mean()
  sdcols = (df[df['year'] == setyear])['NUM_COLLISIONS'].std()
  yearrows = df['year'].value_counts()[setyear]
  print("Calculating year: " + str(setyear))
  print ("--------------------------------------")
  print("Number of rows for year = " + str(yearrows))
  print("Maximum collisions = " + str(maxcols))
  print("Minimum collisions = " + str(mincols))
  print("Mean collisions = " + str(meancols))
  print("sd for collisions = "+ str(sdcols))
  print ("--------------------------------------")

  df.loc[(df['year'] == setyear),'norm_yr'] = ((df['NUM_COLLISIONS'] - mincols) / (maxcols - mincols))
  df.loc[(df['year'] == setyear),'stan_yr'] = ((df['NUM_COLLISIONS'] - meancols) / (sdcols))

# end of Loop


Calculating year: 2012
--------------------------------------
Number of rows for year = 173
Maximum collisions = 718
Minimum collisions = 384
Mean collisions = 554.3988439306358
sd for collisions = 71.04917555815086
--------------------------------------
Calculating year: 2013
--------------------------------------
Number of rows for year = 352
Maximum collisions = 710
Minimum collisions = 393
Mean collisions = 556.1420454545455
sd for collisions = 73.39234024106744
--------------------------------------
Calculating year: 2014
--------------------------------------
Number of rows for year = 352
Maximum collisions = 743
Minimum collisions = 380
Mean collisions = 564.9659090909091
sd for collisions = 78.35149072899698
--------------------------------------
Calculating year: 2015
--------------------------------------
Number of rows for year = 350
Maximum collisions = 798
Minimum collisions = 381
Mean collisions = 596.5742857142857
sd for collisions = 90.96161803635673
-------------------

In [443]:
df

,day,year,mo,da,collision_date,temp,dewp,slp,visib,wdsp,mxpsd,gust,max,min,prcp,sndp,fog,NUM_COLLISIONS,norm_all,stan_all,norm_yr,stan_yr
0,1,2012,7,1,2012-07-01,83.6,63.0,1008.9,9.7,4.1,9.9,18.1,93.0,66.0,0.00,999.9,0,538,0.344227,-0.664699,0.461078,-0.230810
1,3,2012,7,2,2012-07-02,80.3,54.1,1011.6,10.0,3.8,15.0,999.9,88.0,66.9,0.00,999.9,0,564,0.400871,-0.374090,0.538922,0.135134
2,4,2012,7,3,2012-07-03,79.8,56.7,1012.8,10.0,2.9,12.0,999.9,88.0,63.0,0.00,999.9,0,664,0.618736,0.743637,0.838323,1.542610
3,5,2012,7,4,2012-07-04,81.8,65.6,1009.1,9.2,3.6,11.1,999.9,91.0,68.0,0.06,999.9,0,432,0.113290,-1.849489,0.143713,-1.722734
4,6,2012,7,5,2012-07-05,86.7,64.3,1007.4,9.4,3.8,15.0,999.9,93.9,70.0,99.99,999.9,0,591,0.459695,-0.072304,0.619760,0.515152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2732,4,2019,12,24,2019-12-24,42.8,33.7,1015.2,10.0,12.0,18.1,22.0,50.0,34.0,0.00,999.9,0,530,0.326797,-0.754117,0.361111,-0.660921
2734,6,2019,12,26,2019-12-26,35.9,30.3,1025.7,10.0,5.9,14.0,999.9,46.4,28.9,0.00,999.9,0,414,0.074074,-2.050680,0.038889,-2.104749
2735,7,2019,12,27,2019-12-27,40.1,35.0,1026.1,10.0,7.6,12.0,999.9,48.0,28.9,0.00,999.9,0,448,0.148148,-1.670653,0.133333,-1.681558
2738,3,2019,12,30,2019-12-30,39.6,36.3,1017.1,5.2,13.0,25.1,35.0,45.0,30.9,0.39,999.9,0,518,0.300654,-0.888244,0.327778,-0.810283


# 3 Results

## 3.1 Linear regressor

#### 3.1.1 Prepare data


The step below is preparation for inverting the normalisation step, for easier comparison of the collision numbers in the data against the collision numbers produced by the model.

In [444]:
# calculate values required to reverse normalisation of NUM_COLLISIONS

# min
mincolls = min(df["NUM_COLLISIONS"])

# max
maxcolls = max(df["NUM_COLLISIONS"])

# norm_all = (df["NUM_COLLISIONS"]-mincolls) / (maxcolls-mincolls)
#
# therefore
#
# NUM_COLLISIONS = round((norm_all*(maxcolls-mincolls)) + mincolls)

### 3.1.2 Day and number of collisions

For the linear regressor, just the days of the week and the number of collisions is going to be considered first.

In [445]:
# create a dataframe with the inputs and the output at the end using the imported dataframe. This can be replicated for any configuration
df_input_data_day = [df["day"], df["norm_yr"]]
# create headers for our new dataframe. These should correlate with the above.
df_input_headers_day = ["day", "norm_yr"]
# create a final dataframe using our new dataframe and headers.
df_input_day = pd.concat(df_input_data_day, axis=1, keys=df_input_headers_day)

A random selection of 80% of the dataset is set aside to use to train the linear regressor. Of the remaining 20%, a quarter of it is set aside as a validation set (so 5% of the total), leaving 15% to be used as the testing set.

In [446]:
# construct training set with 80% or rows
training_set_day = df_input_day.sample(frac=0.8, random_state=0)

# construct an interim dataframe with remaining 20%
interim_test_set = df_input_day.drop(training_set_day.index)

# construct validation set with 25% of training set (or 5% of total)
validation_set_day = interim_test_set.sample(frac=0.25, random_state=0)

# construct test set with remaining values (15% of total)
test_set_day = interim_test_set.drop(validation_set_day.index)

In [447]:
# copy the datasets and remove the final column, i.e. the output column. We do this using pop.
training_features_day = training_set_day.copy()
test_features_day = test_set_day.copy()

training_labels_day = training_features_day.pop('norm_yr')
test_labels_day = test_features_day.pop('norm_yr')

In [448]:
# boiler plate for this model. You can see that we have used the training_features here for our normalisation layer that we try and fit to the outputs.
normaliser_day = tf.keras.layers.Normalization(input_shape=[1,], axis=None) # tf.keras.layers.Normalization(axis=-1)
normaliser_day.adapt(np.array(training_features_day))


In [449]:
# name the model, model_1, add the normaliser and expect a single output
model_1 = tf.keras.Sequential([
    normaliser_day,
    layers.Dense(units=1)
])

model_1.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [450]:
# fit the model where we require the training features and labels, run it 500 times i.e. epochs and apply a further 20% validation split

%%time
history = model_1.fit(
    training_features_day,
    training_labels_day,
    epochs=500,
    verbose=0,
    validation_split = 0.2)

CPU times: user 1min 1s, sys: 2.92 s, total: 1min 4s
Wall time: 1min


In [451]:
# evaluate model using the test features and labels
mean_absolute_error_model_1 = model_1.evaluate(
    test_features_day,
    test_labels_day, verbose=0)

In [452]:
# print the mean absolute error of the model, which should ideally be minimised, it will also vary on each training run due to randomisation
print(mean_absolute_error_model_1)

0.151498481631279


The above steps were run a handful of times, with the mean absolute error varying between 0.149 and 0.159.

In [453]:
linear_day_predictions_1 = pd.DataFrame((model_1.predict(validation_set_day["day"])*SCALE_NUM_COLLS), columns=['model1'])

5/5 [==============================] - 0s 2ms/step


In [454]:
# append the model1 results to the validation set
validation_set_day_model1 = validation_set_day[["day","norm_yr"]]
validation_set_day_model1['model1'] = linear_day_predictions_1['model1'].values

# reverse the normalisation of NUM_COLLISIONS

validation_set_day_model1['NUM_COLLS'] = round((validation_set_day_model1['norm_yr']*(maxcolls-mincolls)) + mincolls)

validation_set_day_model1['MODEL_COLLS'] = round((validation_set_day_model1['model1']*(maxcolls-mincolls)) + mincolls)

# calculate the percentage of the model result to the number of collisions in the validation set

validation_set_day_model1['%age'] = round((validation_set_day_model1['MODEL_COLLS'] / validation_set_day_model1['NUM_COLLS'] ) * 100)

In [455]:
validation_set_day_model1


,day,norm_yr,model1,NUM_COLLS,MODEL_COLLS,%age
1205,3,0.685851,0.503308,695.0,611.0,88.0
1520,3,0.624352,0.503308,667.0,611.0,92.0
1548,3,0.494819,0.503308,607.0,611.0,101.0
135,4,0.790419,0.545931,743.0,631.0,85.0
2596,2,0.477778,0.460684,599.0,591.0,99.0
...,...,...,...,...,...,...
1530,6,0.738342,0.631178,719.0,670.0,93.0
307,2,0.589905,0.460684,651.0,591.0,91.0
614,7,0.421488,0.673802,573.0,689.0,120.0
922,7,0.887290,0.673802,787.0,689.0,88.0


In the above step, the number of collisions (NUM_COLLS) can be easily compared with the figures produced by the model (MODEL_COLLS). The final column is a percentage value which shows how close the model is. The figures appear reasonable, but the regression model may be improved by adding further predictors. This is undertaken in the next section.

### 3.1.3 Day, dewpoint, temperature and number of collisions

The previous section considered only the day of the week. The upcoming section will consider the day of the week along with two promising weather factors, dewpoint and temperature, to see if it can produce improved modelling.

In [456]:
# create a dataframe with the inputs and the output at the end using the imported dataframe
df_input_data = [df["day"], df["temp"], df["dewp"], df["norm_yr"]]
# create headers for our new dataframe. These should correlate with the above
df_input_headers = ["day", "temp", "dewp", "norm_yr"]
# create a final dataframe using our new dataframe and headers
df_input = pd.concat(df_input_data, axis=1, keys=df_input_headers)

In [457]:
# construct training set with 80% or rows
training_set2 = df_input.sample(frac=0.8, random_state=0)

# construct an interim dataframe with remaining 20%
interim_test2 = df_input.drop(training_set2.index)

# construct validation set with 25% of training set (or 5% of total)
validation_set2 = interim_test2.sample(frac=0.25, random_state=0)

# construct test set with remaining values (15% of total)
test_set2 = interim_test2.drop(validation_set2.index)

In [458]:
# copy the datasets and remove the final column, i.e. the output column. We do this using pop.
training_features = training_set2.copy()
test_features = test_set2.copy()

training_labels = training_features.pop('norm_yr')
test_labels = test_features.pop('norm_yr')

In [459]:
# boiler plate for this model. You can see that we have used the training_features here for our normalisation layer that we try and fit to the outputs.
normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features))

In [460]:
# name the model, model_2, add the normaliser and expect a single output
model_2 = tf.keras.Sequential([
    normaliser,
    layers.Dense(units=1)
])

In [461]:
# more boiler plate for creating a sequential model, need an optimiser and loss parameter, going to be using the mean absolute error MAE
model_2.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [462]:
# fit the model where we require the training features and labels, run it 500 times i.e. epochs and we apply a further 20% validation split

%%time
history = model_2.fit(
    training_features,
    training_labels,
    epochs=500,
    verbose=0,
    validation_split = 0.2)

CPU times: user 1min, sys: 2.92 s, total: 1min 3s
Wall time: 59.2 s


In [463]:
# evaluate the model using the test features and labels
mean_absolute_error_model_2 = model_2.evaluate(
    test_features,
    test_labels, verbose=0)

In [464]:
# print the mean absolute error of the model, which should ideally be minimised, it will also vary on each training run due to randomisation
print(mean_absolute_error_model_2)

0.17403779923915863


The above steps were run a handful of times, with the mean absolute error varying between 0.143 and 0.154, which is only a small reduction on the mean absolute errors found in the section above.


In [465]:
linear_day_predictions_2 = pd.DataFrame((model_2.predict(validation_set2[["day","temp","dewp"]])*SCALE_NUM_COLLS), columns=['model2'])

5/5 [==============================] - 0s 4ms/step


In [466]:
# append the model0 results to the validation set
validation_set_model2 = validation_set2[["day","temp","dewp","norm_yr"]]
validation_set_model2['model2'] = linear_day_predictions_2['model2'].values

# reverse the normalisation of NUM_COLLISIONS

validation_set_model2['NUM_COLLS'] = round((validation_set_model2['norm_yr']*(maxcolls-mincolls)) + mincolls)

validation_set_model2['MODEL_COLLS'] = round((validation_set_model2['model2']*(maxcolls-mincolls)) + mincolls)

# calculate the percentage of the model result to the number of collisions in the validation set

validation_set_model2['%age'] = round((validation_set_model2['MODEL_COLLS'] / validation_set_model2['NUM_COLLS'] ) * 100)


In [467]:
validation_set_model2

,day,temp,dewp,norm_yr,model2,NUM_COLLS,MODEL_COLLS,%age
1205,3,42.8,24.8,0.685851,0.293071,695.0,515.0,74.0
1520,3,72.5,66.3,0.624352,0.612091,667.0,661.0,99.0
1548,3,55.2,41.1,0.494819,0.425304,607.0,575.0,95.0
135,4,52.4,47.5,0.790419,0.486239,743.0,603.0,81.0
2596,2,71.2,62.7,0.477778,0.515097,599.0,616.0,103.0
...,...,...,...,...,...,...,...,...
1530,6,68.5,66.8,0.738342,0.818907,719.0,756.0,105.0
307,2,47.8,38.7,0.589905,0.271887,651.0,505.0,78.0
614,7,29.4,20.9,0.421488,0.488453,573.0,604.0,105.0
922,7,35.2,22.5,0.887290,0.544666,787.0,630.0,80.0


In the above step, the number of collisions (NUM_COLLS) can again be easily compared with the figures produced by the model (MODEL_COLLS). The final column is a percentage value which shows how close the model is. Only a small improvement was achieved by the inclusion of additional predictors (temperature and dewpoint). A deep neural network regressor may achieve better results, and this is explored in the next section.

## 3.2 Deep neural network regressor

### 3.2.1 Prepare data

Firstly, so that the DNN regressor can use the day and month information, additional columns are added for these factors.

In [468]:
# perform one hot encoding of month column
df = pd.get_dummies(df,columns=['mo', ])

# rename month columnns to be more legible
df = df.rename(columns={'mo_1': 'Jan', 'mo_2': 'Feb', 'mo_3': 'Mar', 'mo_4': 'Apr', 'mo_5': 'May','mo_6': 'Jun','mo_7': 'Jul','mo_8': 'Aug','mo_9': 'Sep','mo_10': 'Oct','mo_11': 'Nov','mo_12': 'Dec'})

# perform one hot encoding of day column
df = pd.get_dummies(df,columns=['day', ])

#Rename day columns to be more legible where:
# 1 = Sunday
# 2 = Saturday
# 3 = Monday
# 4 = Tuesday
# 5 = Wednesday
# 6 = Thursday
# 7 = Friday
df = df.rename(columns={'day_1': 'Sun','day_2': 'Sat','day_3': 'Mon','day_4': 'Tue','day_5': 'Wed','day_6': 'Thu','day_7': 'Fri'})


In [469]:
df[:5]

,year,da,collision_date,temp,dewp,slp,visib,wdsp,mxpsd,gust,max,min,prcp,sndp,fog,NUM_COLLISIONS,norm_all,stan_all,norm_yr,stan_yr,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Sun,Sat,Mon,Tue,Wed,Thu,Fri
0,2012,1,2012-07-01,83.6,63.0,1008.9,9.7,4.1,9.9,18.1,93.0,66.0,0.00,999.9,0,538,0.344227,-0.664699,0.461078,-0.230810,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
1,2012,2,2012-07-02,80.3,54.1,1011.6,10.0,3.8,15.0,999.9,88.0,66.9,0.00,999.9,0,564,0.400871,-0.374090,0.538922,0.135134,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
2,2012,3,2012-07-03,79.8,56.7,1012.8,10.0,2.9,12.0,999.9,88.0,63.0,0.00,999.9,0,664,0.618736,0.743637,0.838323,1.542610,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,2012,4,2012-07-04,81.8,65.6,1009.1,9.2,3.6,11.1,999.9,91.0,68.0,0.06,999.9,0,432,0.113290,-1.849489,0.143713,-1.722734,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
4,2012,5,2012-07-05,86.7,64.3,1007.4,9.4,3.8,15.0,999.9,93.9,70.0,99.99,999.9,0,591,0.459695,-0.072304,0.619760,0.515152,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0


### 3.2.2 Day, month, year, dewpoint, temperature and number of collisions

The columns to be used by the deep neural network regressor are defined in the step below.

In [470]:
dnn_input_data = [df["year"], df["temp"], df["dewp"], df["Sat"], df["Sun"], df["Mon"], df["Tue"], df["Wed"], df["Thu"], df["Fri"], df["Jan"], df["Feb"], df["Mar"], df["Apr"], df["May"], df["Jun"], df["Jul"], df["Aug"], df["Sep"], df["Oct"], df["Nov"], df["Dec"], df["norm_all"]]
headers = ["year","temp", "dewp", "Sat","Sun","Mon","Tue","Wed","Thu","Fri","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec","norm_all"]
df_dnn_input = pd.concat(dnn_input_data, axis=1, keys=headers)
df_dnn_input.head()

,year,temp,dewp,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,norm_all
0,2012,83.6,63.0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.344227
1,2012,80.3,54.1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.400871
2,2012,79.8,56.7,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.618736
3,2012,81.8,65.6,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.113290
4,2012,86.7,64.3,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0.459695


In [471]:
# construct training set with 80% or rows
training_dataset = df_dnn_input.sample(frac=0.8, random_state=0)

# construct an interim dataframe with remaining 20%
interim_test_set = df_dnn_input.drop(training_dataset.index)

# construct validation set with 25% of training set (or 5% of total)
validation_dataset = interim_test_set.sample(frac=0.25, random_state=0)

# construct test set with remaining values (15% of total)
test_dataset = interim_test_set.drop(validation_dataset.index)

In [472]:
training_features = training_dataset.copy()
test_features = test_dataset.copy()

training_labels = training_features.pop('norm_all')
test_labels = test_features.pop('norm_all')

In [473]:
normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features))

In [474]:
# this is the only difference, instead of a single layer, we have our normalisation layer (22 inputs), 2 layers of 48, with 1 output. The 48 can be adjusted to improve the net.
dnn_model_1 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_1.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

In [489]:
%%time
history = dnn_model_1.fit(
    training_features,
    training_labels,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 14.3 s, sys: 620 ms, total: 14.9 s
Wall time: 20.5 s


In [490]:
# this should be minimises, the model with the lowest figure is the best
dnn_model_1_results = dnn_model_1.evaluate(test_features, test_labels, verbose=0)
print(dnn_model_1_results)

0.12863261997699738


The above steps were run a number of times, with the mean absolute error varying between 0.114 and 0.129.


In [491]:
linear_day_predictions_dnn = pd.DataFrame((dnn_model_1.predict(validation_dataset[["year","temp", "dewp", "Sat","Sun","Mon","Tue","Wed","Thu","Fri","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]])*SCALE_NUM_COLLS), columns=['dnn_model_1'])


5/5 [==============================] - 0s 4ms/step


In [ ]:
# append the dnn_model_1 results to the validation set
validation_dataset_dnn_model_1 = validation_dataset[["year","temp", "dewp", "Sat","Sun","Mon","Tue","Wed","Thu","Fri","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec","norm_all"]]
validation_dataset_dnn_model_1['dnn_model_1'] = linear_day_predictions_dnn['dnn_model_1'].values

# reverse the normalisation of NUM_COLLISIONS

validation_dataset_dnn_model_1['NUM_COLLS'] = round((validation_dataset_dnn_model_1['norm_all']*(maxcolls-mincolls)) + mincolls)

validation_dataset_dnn_model_1['MODEL_COLLS'] = round((validation_dataset_dnn_model_1['dnn_model_1']*(maxcolls-mincolls)) + mincolls)

# calculate the percentage of the model result to the number of collisions in the validation set

validation_dataset_dnn_model_1['%age'] = round((validation_dataset_dnn_model_1['MODEL_COLLS'] / validation_dataset_dnn_model_1['NUM_COLLS'] ) * 100)

In [ ]:
validation_dataset_dnn_model_1

,year,temp,dewp,Sat,Sun,Mon,Tue,Wed,Thu,Fri,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,norm_all,dnn_model_1,NUM_COLLS,MODEL_COLLS,%age
1205,2015,42.8,24.8,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.625272,0.651899,667.0,679.0,102.0
1520,2016,72.5,66.3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.638344,0.578252,673.0,645.0,96.0
1548,2016,55.2,41.1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.529412,0.674983,623.0,690.0,111.0
135,2012,52.4,47.5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.583878,0.514852,648.0,616.0,95.0
2596,2019,71.2,62.7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.418301,0.350528,572.0,541.0,95.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,2016,68.5,66.8,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0.734205,0.737070,717.0,718.0,100.0
307,2013,47.8,38.7,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.435730,0.279986,580.0,509.0,88.0
614,2014,29.4,20.9,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0.333333,0.485263,533.0,603.0,113.0
922,2015,35.2,22.5,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0.808279,0.478778,751.0,600.0,80.0


In the above step, the number of collisions (NUM_COLLS) can again be easily compared with the figures produced by the model (MODEL_COLLS). The final column is a percentage value which shows how close the model is. The predictors used for the DNN regressor are day, month, year, temperature and dewpoint. The DNN regression model's results are better than the results of the linear regression models, as evidenced by the consistently lower mean absolute error.

# 4 Conclusion

The first linear regression model developed above, which included only day of the week and numbers of collisions, produced a lowest mean absolute error (MAE) of 0.149 when run a handful of times.  Clearly, this could be improved upon. The second linear regression model used dewpoint and temperature as well as day, and led to a small improvement in which the lowest MAE produced was 0.143.

Subsequently, a deep neural network (DNN) model was developed, which included day, month, year, dewpoint, temperature and number of collisions. This model was found to be better at predicting collision numbers than the linear regression models, supported by a reduced MAE which was 0.114 at its lowest.

While the specific mean absolute error figures will differ each time the models are run, largely because the training sets are randomised, it is reasonable to conclude that the DNN regressor is more reliable that the linear regressors, albeit a proportion of that improvement will have occurred due to the inclusion of additional predictors.
